[Source](https://https://ebookcentral.proquest.com/lib/uncw/reader.action?docID=30722783&ppg=13)

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] ="API_KEY" # Add your API Key here :)


from openai import OpenAI

client = OpenAI()

# Make sure the environment variable OPENAI_API_KEY is set.

# Call the openai ChatCompletion endpoint, with th ChatGPT model
response = client.chat.completions.create(model="gpt-3.5-turbo",
messages=[
      {"role": "user", "content": "Hello World!"}
  ])

# Extract the response
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [ ]:
import json

In [ ]:
def find_product(sql_query):
    # Execute query here
    results = [
        {"name": "pen", "color": "blue", "price": 2.99}, # Dictionary, json database
        {"name": "pencil", "color": "purple", "price": 1.99},
        {"name": "tape", "color": "light blue", "price": 5.99},
        {"name": "nickel", "color": "silver", "price": 0.05}
    ]
    return results


function_find_product = {
    # Gets a list of products with an SQL query
        "name": "find_product",
        "description": "Get a list of products from a sql query",
        "parameters": {
            "type": "object",
            "properties": {
                "sql_query": {
                    "type": "string",
                    "description": "A SQL query",
                }
            },
            "required": ["sql_query"],
        },
    }



def run(user_question):
    # Send the question and available functions to GPT
    messages = [{"role": "user", "content": user_question}]

    # Here, we're providing the function name
    # Previously, we have used prompts to ask the chatbot for a response
    response = client.chat.completions.create(model="gpt-3.5-turbo-0613", messages=messages, tools=[{"type": "function", "function": function_find_product }])
    response_message = response.choices[0].message # Sending message to client

    # Append the assistant's response to the messages
    messages.append(response_message) # This is the model's message


    # Call the function and add the results to the messages
    function_name = response_message.tool_calls[0].function.name
    if function_name == "find_product":
        function_args = json.loads(
            response_message.tool_calls[0].function.arguments
        )
        products = find_product(function_args.get("sql_query")) # Running an SQL query in the background
        print(function_args.get('sql_query'))
    else:
        # Handle error
        products = []
    # Append the function's response to the messages
    messages.append(
        {
            "role": "tool",
            "content": json.dumps(products),
            "tool_call_id": response_message.tool_calls[0].id,
        }
    )
    # Get a new response from GPT so it can format the function's response into natural language
    # Asking GPT to summarize response again in a user-friendly way
    second_response = client.chat.completions.create(model="gpt-3.5-turbo-0613",
    messages=messages)
    return second_response.choices[0].message.content


print(run("I need the top 3 products where the price is greater than 2.00"))

SELECT * FROM products WHERE price > 2.00 ORDER BY price DESC LIMIT 3
The top 3 products with a price greater than $2.00 are:

1. Tape - Price: $5.99
2. Pen - Price: $2.99
3. Pencil - Price: $1.99
